# Generator

In [2]:
%matplotlib inline
import numpy as np
import os
from timeit import default_timer as timer
import image_tools as imt
from augmentator import Augmentator
from generator import Generator

In [3]:
image_shape = [128,128,1]
images_dir  = "/netforge/datasets/private/roman/coins/2017-11-29/images/train/"
dataset_dir = "/netforge/datasets/private/roman/coins/2017-11-29/datasets/train/"

## Generate

In [7]:
%%time 
gen = Generator()
try:
    start = timer()
    gen.generate(
        images_path = images_dir, 
        image_shape = image_shape, 
        augmentator = Augmentator(), 
        image_num   = 1000,
        aug_num     = 300,
    )
    t = timer() - start
    augmented = gen.augmented
    n = augmented.shape[0]*augmented.shape[1]
    print("Augmented  : %d images" % n)
    print("Performance: %.1f img/sec" % (n/t))
    print("Augmented  :", augmented.dtype, augmented.shape, augmented.nbytes/1024/1024, "MB")
except KeyboardInterrupt:
    print("Interrupted")
print("-"*50)

100%|██████████| 300/300 [38:37<00:00,  7.72s/it]


Augmented  : 300000 images
Performance: 129.4 img/sec
Augmented  : uint8 (1000, 300, 128, 128, 1) 4687.5 MB
--------------------------------------------------
CPU times: user 30min 10s, sys: 25min 57s, total: 56min 8s
Wall time: 38min 39s


## Save

In [8]:
%%time
dataset_path = gen.save(dataset_dir)
print("Dataset:", dataset_path)

Dataset: /netforge/datasets/private/roman/coins/2017-11-29/datasets/train/1000_300_128_128_1_uint8.npy
CPU times: user 384 ms, sys: 2.69 s, total: 3.07 s
Wall time: 6.04 s


## Load

In [ ]:
%%time
try:
    del gen.data
    del gen
except:
    pass
dataset_path = "/netforge/datasets/private/roman/coins/2017-11-29/datasets/train/5557_300_128_128_1_uint8.npy"
data = np.load(dataset_path)
print("Data:", data.dtype, data.shape, data.nbytes/1024/1024, "MB")
print("-"*50)

## Show

In [ ]:
images = data[7,:42]
images = imt.uint8_to_float32(images)
imt.plot(images, 6)